In [1]:
import os
import time
import torch
import sys
import torch.optim as optim
from torchvision import transforms as T
%matplotlib notebook

In [2]:
from models import SpecialFuseNetModel
from data_manager import rgbd_gradients_dataset, rgbd_gradients_dataloader
from train import FuseNetTrainer
from plot import post_epoch_plot

In [3]:
CWD             = os.getcwd()
DATASET_DIR     = os.path.join(CWD,'data/nyuv2')
print(DATASET_DIR)

C:\Users\tomav\Documents\GitHub\cs236781-project\data/nyuv2


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [5]:
rgbd_grads_ds = rgbd_gradients_dataset(root=DATASET_DIR, use_transforms=True)

TypeError: __init__() missing 1 required positional argument: 'image_size'

In [ ]:
dl_train,dl_test = rgbd_gradients_dataloader(root=DATASET_DIR,use_transforms=True)

In [ ]:
# _ = plot.rgbd_gradients_dataset_first_n(dataset=rgbd_grads_ds,n=5)
print(f'Found {len(rgbd_grads_ds)} images in dataset folder.')

In [ ]:
sample_batch = next(iter(dl_train))
rgb_size = tuple(sample_batch['rgb'].shape[1:])
depth_size = tuple(sample_batch['depth'].shape[1:])
grads_size = tuple(sample_batch['x'].shape[1:])

In [ ]:
fusenetmodel = SpecialFuseNetModel(rgb_size=rgb_size,depth_size=depth_size,grads_size=grads_size, device=device)

In [ ]:
trainer = FuseNetTrainer(model=fusenetmodel, device=device)

In [ ]:
checkpoint_file = 'checkpoints/special_fusenet'
checkpoint_file_final = "checkpoints/special_fusenetimg_size=64,64,batch_size=4,betas=0.9,0.99,lr=0.001,momentum=0.9"
if os.path.isfile(f'{checkpoint_file}.pt'):
    print(f'[I] - {checkpoint_file} exist')
    os.remove(f'{checkpoint_file}.pt')
if os.path.isfile(f'{checkpoint_file_final}.pt'):
    print(f'[I] - {checkpoint_file_final} exist')

In [ ]:
res = trainer.fit(dl_train, dl_test, early_stopping=20, print_every=10,
                  checkpoints=checkpoint_file_final, post_epoch_fn=post_epoch_plot)